# Web Scraper code
For web scraping I have used selenium along with beutiful soup. Selenium was necessary according to the URL that has been provided. The data that we want is spread over 711 pages with 50 rows on each page except the last page. The URL for each page is also the same. So even if we copy pasted the URL when we're viewing the 100th page, in a new tab that would still show us the first page itself. 
> Thus, its necessary to iterate through each page of the table. For this automation process, as we want to view the table after clicking the next button I have used selenium.

In [ ]:
# installing the necessary packages
!pip install selenium
!pip install beautifulsoup4 
!pip install webdriver-manager      # this makes it easier to load browser configurations for selenium

As there are 711 pages, we need to get the data from each page then click the next button and wait till the next page loads. Then we get the data from that page and continue with the same process. 
Some important points of the source that we are scraping from:
* URL for each page of the table is the same, hence looping through URL values isn't possible
* Note that the first page has only 2 buttons i.e. 'Next' and 'Last' & the last page has 'Prev' and 'First'
* Every other page has 4 buttons i.e. 'Next' 'Last' 'Prev' 'First'
* After any of the buttons is clicked the website takes some time to show the result of the same, about 10-15 seconds

Keeping these points in mind, we deal with the first page separately and then the rest of them through a loop which iterates 710 times. I have explicitly put a sleep of 20 seconds wherever necessary so that the page has been loaded before we try to extract the data. I am finding the button and the table using selectors that I got from inspect element of the URL.

> The detailed reason for keeping the first page different is that the selector for the 'Next' button on the first page and pages after that are different.

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

link = "https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=78&Tx_State=KK&Tx_District=0&Tx_Market=0&DateFrom=01-Jan-2015&DateTo=01-Feb-2021&Fr_Date=01-Jan-2015&To_Date=01-Feb-2021&Tx_Trend=0&Tx_CommodityHead=Tomato&Tx_StateHead=Karnataka&Tx_DistrictHead=--Select--&Tx_MarketHead=--Select--"

with webdriver.Chrome(ChromeDriverManager().install()) as driver:
    driver.get(link)

    datalist = []
    
    wait = WebDriverWait(driver, 10)
    
    x = 1   # page counter
    
    if x == 1:
        show_more = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"#cphBody_GridPriceData > tbody > tr:nth-child(52) > td > table > tbody > tr > td:nth-child(1) > input[type=image]")))
        show_more.click()
        time.sleep(20)

        for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"#cphBody_GridPriceData tr"))):
            data = [item.text for item in elem.find_elements_by_css_selector("td")]
            datalist.append(data)
        x = x + 1
    
    while x < 712:
        show_more = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"#cphBody_GridPriceData > tbody > tr:nth-child(52) > td > table > tbody > tr > td:nth-child(3) > input[type=image]")))
        show_more.click()
        time.sleep(20)

        for elem in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"#cphBody_GridPriceData tr"))):
            data = [item.text for item in elem.find_elements_by_css_selector("td")]
            datalist.append(data)
        x = x + 1

In [ ]:
# convert to pandas dataframe
df = pd.DataFrame(datalist)
print(df)

In [ ]:
# convert to csv format
df.to_csv('Agri_data.csv')